<a href="https://colab.research.google.com/github/Alex-Ukraine/ml-selfedu/blob/main/deepRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
from tensorflow.keras.layers import Dense, SimpleRNN, Input, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

from google.colab import files
files.upload()

Saving text to text


{'text': b'\xef\xbb\xbf\xd0\xaf \xd0\xbf\xd1\x80\xd0\xb8\xd1\x82\xd1\x8f\xd0\xb3\xd0\xb8\xd0\xb2\xd0\xb0\xd1\x8e \xd1\x82\xd0\xbe\xd0\xbb\xd1\x8c\xd0\xba\xd0\xbe \xd0\xbf\xd0\xbb\xd0\xbe\xd1\x85\xd0\xbe\xd0\xb5\r\n\xd0\x9a\xd0\xbe\xd0\xbc\xd1\x83 \xd1\x8f \xd0\xbd\xd1\x83\xd0\xb6\xd0\xb5\xd0\xbd \xd1\x81 \xd1\x82\xd0\xb0\xd0\xba\xd0\xbe\xd0\xb9 \xd0\xb2\xd0\xbd\xd0\xb5\xd1\x88\xd0\xbd\xd0\xbe\xd1\x81\xd1\x82\xd1\x8c\xd1\x8e\r\n\xd0\x9d\xd0\xb5 \xd0\xb4\xd0\xbe\xd0\xb2\xd0\xb5\xd1\x80\xd1\x8f\xd0\xb9 \xd0\xbd\xd0\xb8\xd0\xba\xd0\xbe\xd0\xbc\xd1\x83, \xd1\x82\xd0\xb5\xd0\xb1\xd1\x8f \xd0\xbe\xd0\xb1\xd1\x8f\xd0\xb7\xd0\xb0\xd1\x82\xd0\xb5\xd0\xbb\xd1\x8c\xd0\xbd\xd0\xbe \xd0\xbe\xd0\xb1\xd0\xbc\xd0\xb0\xd0\xbd\xd1\x83\xd1\x82\r\n\xd0\x95\xd1\x81\xd0\xbb\xd0\xb8 \xd1\x80\xd0\xb8\xd1\x81\xd0\xba\xd0\xbd\xd0\xb5\xd1\x88\xd1\x8c \xd0\xb2\xd1\x8b\xd1\x81\xd1\x83\xd0\xbd\xd1\x83\xd1\x82\xd1\x8c\xd1\x81\xd1\x8f, \xd1\x82\xd0\xbe \xd0\xb4\xd0\xbe\xd0\xb1\xd1\x80\xd0\xbe\xd0\xbc \xd1\x8d\xd1\x82\

In [ ]:
with open('text', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ

In [ ]:
maxWordsCount = 1000
tokenizer=Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', 
                    lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([text]) # формируем токены на основе частотности в нашем тексте

In [ ]:
# для примера выведем некие слова и их частотность
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('я', 21), ('притягиваю', 1), ('только', 21), ('плохое', 1), ('кому', 2), ('нужен', 2), ('с', 12), ('такой', 5), ('внешностью', 2), ('не', 91)]


In [ ]:
data = tokenizer.texts_to_sequences([text])
# res = to_categorical(data[0], num_classes=maxWordsCount)
# print(res.shape)
res = np.array( data[0] )

In [ ]:
inp_words = 3
n = res.shape[0] - inp_words

# list comprehentions
X = np.array([res[i:i + inp_words] for i in range(n)])
#Y = res[inp_words:] # предсказание следующего слова
Y = to_categorical(res[inp_words:], num_classes=maxWordsCount)

In [ ]:
model = Sequential()
# входной слой имеет такой формат, это тензор (batch_size, inp_chars, num_characters)
#model.add(Input((inp_words,
#                 maxWordsCount))) # при тренировке в рекурентные модели keras подаётся сразу вся последовательность 
model.add(Embedding(maxWordsCount, 256, input_length = inp_words))
model.add(SimpleRNN(128, activation='tanh', return_sequences=True)) # благодаря return_sequences - 2 рекк-ых слоя
model.add(SimpleRNN(64, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3, 256)            256000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 3, 128)            49280     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                12352     
_________________________________________________________________
dense (Dense)                (None, 1000)              65000     
Total params: 382,632
Trainable params: 382,632
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
52/52 [==============================] - 0s 5ms/step - loss: 6.8205 - accuracy: 0.0294
Epoch 2/50
52/52 [==============================] - 0s 5ms/step - loss: 6.1512 - accuracy: 0.0559
Epoch 3/50
52/52 [==============================] - 0s 5ms/step - loss: 5.7654 - accuracy: 0.0691
Epoch 4/50
52/52 [==============================] - 0s 5ms/step - loss: 5.3306 - accuracy: 0.1244
Epoch 5/50
52/52 [==============================] - 0s 5ms/step - loss: 4.8597 - accuracy: 0.2115
Epoch 6/50
52/52 [==============================] - 0s 5ms/step - loss: 4.4042 - accuracy: 0.3131
Epoch 7/50
52/52 [==============================] - 0s 5ms/step - loss: 3.9737 - accuracy: 0.3990
Epoch 8/50
52/52 [==============================] - 0s 5ms/step - loss: 3.5692 - accuracy: 0.4688
Epoch 9/50
52/52 [==============================] - 0s 5ms/step - loss: 3.1943 - accuracy: 0.5643
Epoch 10/50
52/52 [==============================] - 0s 5ms/step - loss: 2.8508 - accuracy: 0.6376
Epoch 11/50
52/52 [

In [ ]:
def buildPhrase(texts, str_len=20):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        #x = to_categorical(data[i: i + inp_words], num_classes=maxWordsCount) # преобразовывает в One-Hot-Encoding
        # формируем слова, на основе которорых делаем прогноз следующего слова
        #inp = x.reshape(1, inp_words, maxWordsCount)
        x = data[i: i + inp_words]
        inp = np.expand_dims(x, axis=0)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx] # дописываем строку

    return res

In [ ]:
res = buildPhrase("позитив добавляет годы")
print(res)

позитив добавляет годы счастье вашей жизни и двигаться их в вы держись в это мечты и без заставить если у не есть память


Embedding позволяет сократить расходы памяти

keras.layers.Embedding(input_dim, output_dim, ..., input_length)

* input_dim - число слов в словаре;
* outputdim - число выходов в полносвязном Embedding-слое;
* input_length - размер входного вектора (число слов inp_words, по которым строиться прогноз)